In [1]:
# 引入需要的包裝

import sys
import pyvisa
import time
from PyQt6 import QtWidgets
from UI import Ui_MainWindow
from MSO64B import MSO_64B  # 匯入 MSO_64B 類

In [2]:
class MSO_64B():
    def __init__(self, resource_address, visa_dll=None):
        """
        初始化儀器控制器。
        :param resource_address: 儀器的資源地址。
        :param visa_dll: 使用的 VISA DLL 路徑。如果為 None，則使用系統預設空白。
        """
        self.resource_address = resource_address
        self.visa_dll = visa_dll if visa_dll is not None else ''
        self.rm = pyvisa.ResourceManager(self.visa_dll)
        self.instrument = None
        self.connect_to_instrument()
        
    def connect_to_instrument(self):
        # 建立與儀器的連接
        try:
            self.instrument = self.rm.open_resource(self.resource_address)
            print("Connected to MSO64B.")
            IDN = self.query_identity()
            print(IDN)
        except Exception as e:
            print(f"Failed to connect to instrument: {e}")
            self.instrument = None
        
    def query_identity(self):
         # 查詢並返回儀器的身份識別信息
        if self.instrument:
             return self.instrument.query("*IDN?")
        else:
             raise Exception("Instrument not connected. Please connect first.")

    def close_instrument(self):
        if self.instrument:
            self.instrument.close()
            print("MSO64B connection closed.")
        else:
            print("No instrument to close.")

    def display_channel(self, channel, state):
        #將通道顯示在示波器畫面
        if self.instrument:
            self.instrument.write(f':DIS:WAVEVIEW1:{channel}:STATE {state}') 
            time.sleep(0.05)  # 在命令之間暫停0.5秒
            print(f"Turn {channel} {state}")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def set_acquistion(self, acquistion_num):
        #設定採樣模式及數量
        if self.instrument:
            self.instrument.write(f':ACQuire:MODe AVE;:ACQuire:NUMAVg {acquistion_num}')
            #設定Acquistion mode為average並波形獲取數量為128
            time.sleep(0.05)  # 在命令之間暫停0.5秒
            print(f"set acquisition in average mode and waveform acquire number be {acquistion_num}")
        else:
            raise Exception("Instrument not connected. Please connect first.") 

    def probe_autozero(self, channel):
        #將探棒Auto Zero調整波形位準
        if self.instrument:
            self.instrument.write(f':{channel}:PRObe:AUTOZero EXECute')
            time.sleep(0.05)  
            print(f"set {channel} Auto Zero")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def probe_bandwidth_limit(self, channel, limit):
        #設定探棒的頻寬限制
        if self.instrument:
            self.instrument.write(f':{channel}:BANdwidth {limit}')
            print(f"set {channel} Bandwidth Limit be {limit}E+06")
        else:
            raise Exception("Instrument not connected. Please connect first.") 

In [3]:
class MainWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)
        self.mso = None  # 儀器實例變量
        self.setup_control()

    def setup_control(self):
        # 連接按鈕
        self.ui.address_refresh.clicked.connect(self.refresh_instruments)
        self.ui.connect_button.clicked.connect(self.connect_instruments)
        
    def refresh_instruments(self):
        """刷新可用的儀器地址並更新 QComboBox"""
        try:
            rm = pyvisa.ResourceManager()
            instrument_list = rm.list_resources()

            # 清空現有選項
            self.ui.Scope_IDN.clear()
            self.ui.Scope_address.clear()
            self.ui.Scope_address.addItem("None") #新增一個none項目

            # 添加新選項到 ComboBox
            for address in instrument_list:
                self.ui.Scope_address.addItem(address)

            self.statusBar().showMessage("Instrument addresses refreshed.")
        
        except Exception as e:
            self.statusBar().showMessage(f"Failed to refresh instruments: {str(e)}")

    def connect_instruments(self):
        status_message = ""  # 用來收集所有的連接狀態訊息
        
        # 連接示波器 MSO 64B
        mso_address = self.ui.Scope_address.currentText()
        if mso_address == "None" or not mso_address:
            status_message += "Failed connedct to Scope, Please select a valid instrument address.\n"
        else:
            try:
                self.mso = MSO_64B(mso_address)
                mso_idn = self.mso.query_identity()  # 查詢 IDN
                self.ui.Scope_IDN.setText(mso_idn)
                status_message += f"Scope connected to {mso_idn}.\n"
            except Exception as e:
                status_message += f"Failed to connect to Scope: {str(e)}.\n"

        # 最後一次性更新 statusBar
        self.statusBar().showMessage(status_message)


In [4]:
def main():
    app = QtWidgets.QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec())

if __name__ == "__main__":
    main()

Connected to MSO64B.
TEKTRONIX,MSO64B,C048992,CF:91.1CT FV:1.44.3.433

set acquisition in average mode and waveform acquire number be 64.0


SystemExit: 0

C:\Users\User\anaconda3\envs\PyVISA\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
